In [5]:
import os
os.environ['https_proxy'] = 'http://192.41.170.23:3128'
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
import random
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("kamalkraj/BioELECTRA-PICO")
model = AutoModelForTokenClassification.from_pretrained("kamalkraj/BioELECTRA-PICO")

model.to(device)

data = pd.read_csv("/home/st122802/work/Thesis/Untitled_Folder/cleaned_train-targets.csv")

def preprocess_background(background):
    background = background.replace("(", "").replace(")", "").replace(",", "").replace("\n", " ")
    inputs = tokenizer.encode(background, return_tensors="pt", truncation=True, max_length=512).to(device)
    
    outputs = model(inputs).logits
    predictions = outputs.argmax(dim=2)[0]

    tokens = tokenizer.convert_ids_to_tokens(inputs[0])
    
    masked_tokens = []
    for token, prediction in zip(tokens, predictions):
        label = model.config.id2label[prediction.item()]

        if label != 'O' and random.random() < 0.45:
            token = tokenizer.mask_token
            
        if token not in ['[CLS]', '[SEP]']:
            masked_tokens.append(token)

    return " ".join(masked_tokens)

data['Masked Background'] = data['Background'].apply(preprocess_background)

data.to_csv("/home/st122802/work/Thesis/Untitled_Folder/Final training sample/masked-background_PICO_0.45.csv", index=False)

In [13]:
import pandas as pd
import random

input_file_path = '/home/st122802/work/Thesis/Untitled Folder/train-targets.csv'

data = pd.read_csv(input_file_path)


mask_ratios = [0.15, 0.30, 0.45]

for ratio in mask_ratios:
    output_file_path = f'masked_background_random_{int(ratio)}.csv' 
    
    masked_data = data.copy() 
    
    for index, row in masked_data.iterrows():
        background_text = row['Background']
        words = background_text.split()
        num_words_to_mask = max(1, int(len(words) * ratio))
        masked_indices = random.sample(range(len(words)), num_words_to_mask)

        for idx in masked_indices:
            words[idx] = '[MASK]'

        masked_text = ' '.join(words)
        masked_data.at[index, 'Background'] = masked_text

  
    masked_data.to_csv(output_file_path, index=False)